In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
spark = SparkSession.builder.appName('recommend').getOrCreate()

df = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [4]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train, test = df.randomSplit([0.8, 0.2])

als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [8]:
als_model = als.fit(train)

predictions = als_model.transform(test)
predictions.show()

22/02/21 12:15:52 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/02/21 12:15:52 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/02/21 12:15:53 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      1|   1.0|     4|   1.7668525|
|      1|   1.0|    14|   1.7228354|
|      6|   1.0|     4|   1.3808091|
|      6|   2.0|    11|   1.2650586|
|      6|   1.0|    13|  0.72865504|
|      3|   1.0|     0|-0.006935926|
|      3|   1.0|     9|   1.5665396|
|      3|   1.0|    13|  0.37420997|
|      3|   1.0|    29|  0.92283666|
|      5|   1.0|    13|   0.7763567|
|      5|   2.0|    15|   1.5594337|
|      5|   2.0|    22|   -0.497984|
|      4|   1.0|     5|   1.0023692|
|      4|   1.0|    23|  0.06425604|
|      7|   1.0|     7|   1.4593964|
|      2|   3.0|     9|   1.6061729|
|      2|   2.0|    20|   3.1050591|
|      2|   4.0|    28|   2.9098022|
|      0|   1.0|    19|  0.26905215|
|      0|   1.0|    26|   2.8717215|
+-------+------+------+------------+
only showing top 20 rows



In [9]:
#ealuate the model

evaluator  = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)

print('RMSE {}'.format(rmse))

RMSE 1.713332054174349


In [10]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     19|    11|
|     32|    11|
|     40|    11|
|     43|    11|
|     45|    11|
|     48|    11|
|     66|    11|
|     69|    11|
|     75|    11|
|     80|    11|
|     81|    11|
|     86|    11|
|     89|    11|
+-------+------+



In [12]:
results = als_model.transform(single_user)
results.orderBy('prediction', ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     32|    11|   4.8781657|
|     66|    11|    1.974521|
|     81|    11|     1.79212|
|     43|    11|   1.4767618|
|     48|    11|   1.3528593|
|     86|    11|   1.2748035|
|      6|    11|   1.2650586|
|     19|    11|   1.0810251|
|     75|    11|  0.84736913|
|     40|    11|  0.46531785|
|     80|    11|-0.063049436|
|     69|    11| -0.32325613|
|     89|    11|  -0.7662194|
|     45|    11| -0.77860826|
+-------+------+------------+



In [13]:
#thanks for going through this notebook